In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical,pad_sequences
from keras.models import load_model

In [22]:
df = pd.read_csv('..\\dataset\\spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [23]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [24]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [25]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [26]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [27]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [28]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [29]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 9s 170ms/step - loss: 0.3181 - accuracy: 0.8831 - val_loss: 0.1487 - val_accuracy: 0.9536
Epoch 2/10
30/30 [==============================] - 5s 150ms/step - loss: 0.0862 - accuracy: 0.9791 - val_loss: 0.0945 - val_accuracy: 0.9715
Epoch 3/10
30/30 [==============================] - 4s 147ms/step - loss: 0.0474 - accuracy: 0.9860 - val_loss: 0.0571 - val_accuracy: 0.9852
Epoch 4/10
30/30 [==============================] - 4s 146ms/step - loss: 0.0374 - accuracy: 0.9894 - val_loss: 0.0529 - val_accuracy: 0.9873
Epoch 5/10
30/30 [==============================] - 4s 148ms/step - loss: 0.0276 - accuracy: 0.9923 - val_loss: 0.0534 - val_accuracy: 0.9873
Epoch 6/10
30/30 [==============================] - 4s 146ms/step - loss: 0.0216 - accuracy: 0.9934 - val_loss: 0.0576 - val_accuracy: 0.9873
Epoch 7/10
30/30 [==============================] - 4s 149ms/step - loss: 0.0169 - accuracy: 0.9950 - val_loss: 0.0633 - val_accuracy: 0.9873
Epoch 

In [30]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 19ms/step - loss: 0.0599 - accuracy: 0.9868


[0.05993932858109474, 0.9868420958518982]

In [31]:
model.save("..\\models\\sms.keras")

In [32]:
model1 = load_model("..\\models\\sms.keras")

In [33]:
model1.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 21ms/step - loss: 0.0599 - accuracy: 0.9868


[0.05993932858109474, 0.9868420958518982]

In [34]:
import pickle

# saving tokenizer
with open('..\\models\\smstoken.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
with open('..\\models\\smstoken.pickle', 'rb') as handle:
    t = pickle.load(handle)

In [43]:
max_words = 1000
max_len = 150
text=["you won money, fill this form","time to die","how are you"]
df1=pd.DataFrame(text,columns=["text"])
x=df1["text"]
sequences = t.texts_to_sequences(x)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)
model1.predict(sequences_matrix)

1/1 [==============================] - 0s 40ms/step


array([[0.2547036 ],
       [0.00340138],
       [0.00123494]], dtype=float32)